# Import Libraries

In [1]:
import PyPDF2 as pypdf2
import fitz
import pandas as pd

## Define Functions

In [2]:
# Extract textual data from PDF
def extract_text_from_pdf(pdf_path, start_page, end_page, package):
    text = '';
    
    if package == "pypdf2":
        with open(pdf_path, 'rb') as file:
            pdf_reader = pypdf2.PdfReader(file);

            for page_num in range(start_page, end_page+1):
                page = pdf_reader.getPage(page_num);
                page_text = page.extractText();
                text += page_text;
    elif package == "fitz":
        pdf_document = fitz.open(pdf_path);

        for page_num in range(start_page, end_page+1):
            page = pdf_document.load_page(page_num);
            page_text = page.get_text();
            text += page_text;
    
    return text;

In [3]:
# Get section names & page numbers from table of contents of PDF
def get_section_and_pages(unclean_text):
    unclean_text_list = unclean_text.split("\n")[5:13];
    rough_list = [];
    for text in unclean_text_list:
        if "p." not in text:
            if "1." not in text:
                text = text.replace("p","p.");
                rough_list.append(text);
            else:
                rough_list.append(unclean_text_list[0] + unclean_text_list[1]);
        else:
            rough_list.append(text);
    rough_list.pop(1);

    clean_list = [];
    for text in rough_list:
        rough_section = text.split("p.")[0];
        rough_pages = str(text.split("p.")[1]);

        section = rough_section.replace(","," ").replace("  "," ").rstrip();
        
        pages = rough_pages.strip();
        if "-" in rough_pages:
            pages = rough_pages.replace(" -","-").strip();
        if "." in rough_pages:
            pages = rough_pages.replace(".","-").strip();

        start_page = pages;
        end_page = pages;
        if "-" in pages:
            page_range = pages.split("-");
            start_page = int(page_range[0]);
            end_page = int(page_range[1]);
            if section == "3.Health & Safety in boutique Level 1":
                end_page = end_page + 1;
            elif section == "5.Food Safety & Quality Level 1":
                start_page = start_page + 11;
                end_page = start_page + 9;
            elif section == "6.Stock Management":
                end_page = end_page - 1;
        else:
            if section == "7.Sustainability in BTQ":
                start_page = int(pages);
                end_page = int(pages) + 1;

        clean_list.append({
            "section":section,
            "start_page":start_page,
            "end_page":end_page
        });    

    return clean_list;

In [4]:
# Extract text from PDF per section (i.e., within range of start page & end page), and save in a dataframe
def save_data_in_dataframe(pdf_path, dict_list, col_list, package):
    data = [];
    for elem in dict_list:
        text = extract_text_from_pdf(pdf_path, elem.get("start_page"), elem.get("end_page"), package);
        row = [elem.get("section"), elem.get("start_page"), elem.get("end_page"), text];
        data.append(row);
    df = pd.DataFrame(data, columns=col_list);
    return df;

# Instantiate Constant Variables

In [5]:
# The pdf document whereby text will be extracted from
pdf_path = "00_Security Health  Safety Sustainability MEMO for ALL BTQ Employees _ 2022_V2.pdf";

# Dictionary list containing section name, as well as start page and end page of respective section
section_pages_list = get_section_and_pages(unclean_text=extract_text_from_pdf(pdf_path, 1, 1, "pypdf2"));

# Filename for extracted Raw data that will be saved in Excel workbook
filename = "RawData.xlsx";

# Extract Data from PDF and save in Dataframe

In [6]:
# Instantiate dataframe
df = save_data_in_dataframe(
    pdf_path=pdf_path, 
    dict_list=section_pages_list, 
    col_list=["Section","Start Page","End Page","Extracted Text"],
    package="fitz"
);

# General information regarding dataframe
print(df.info());

# Show dataframe
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Section         7 non-null      object
 1   Start Page      7 non-null      int64 
 2   End Page        7 non-null      int64 
 3   Extracted Text  7 non-null      object
dtypes: int64(2), object(2)
memory usage: 352.0+ bytes
None


,Section,Start Page,End Page,Extracted Text
0,1.Nestlé Nespresso Quality and SHE policies –Q...,3,6,"QUALITY, SECURITY\nHEALTH & SAFETY \nSUSTAINAB..."
1,2.Emergency Preparedness and Response,7,11,"QUALITY, SECURITY\nHEALTH & SAFETY \nSUSTAINAB..."
2,3.Health & Safety in boutique Level 1,12,20,"QUALITY, SECURITY\nHEALTH & SAFETY \nSUSTAINAB..."
3,4.Security Level 1,21,31,"QUALITY, SECURITY\nHEALTH & SAFETY \nSUSTAINAB..."
4,5.Food Safety & Quality Level 1,32,41,"QUALITY, SECURITY\nHEALTH & SAFETY \nSUSTAINAB..."
5,6.Stock Management,42,45,"QUALITY, SECURITY\nHEALTH & SAFETY, \nSUSTAINA..."
6,7.Sustainability in BTQ,46,47,"QUALITY, SECURITY\nHEALTH & SAFETY, \nSUSTAINA..."


In [9]:
# Save dataframe to Excel workbook
df.to_excel(filename, index=False);